In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,39519
2,Huelva,Confirmados PDIA 14 días,3120
3,Huelva,Tasa PDIA 14 días,"607,9856577742269"
4,Huelva,Confirmados PDIA 7 días,1481
5,Huelva,Tasa PDIA 7 dias,"288,5983202447532"
6,Huelva,Total Confirmados,39726
7,Huelva,Curados,34494
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  39519.0


/tmp/ipykernel_3320/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11073.0


/tmp/ipykernel_3320/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3320/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3320/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3320/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 315 personas en los últimos 7 días 

Un positivo PCR cada 152 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,39519.0,1481.0,3120.0,513170.0,288.598320,607.985658,186.0
Huelva-Costa,23241.0,910.0,1921.0,289548.0,314.282951,663.447857,109.0
Condado-Campiña,12370.0,431.0,821.0,156231.0,275.873546,525.503901,58.0
Huelva (capital),11073.0,456.0,946.0,143837.0,317.025522,657.688912,53.0
Punta Umbría,1111.0,78.0,160.0,15355.0,507.977857,1042.005861,20.0
Sierra de Huelva-Andévalo Central,3529.0,135.0,372.0,67391.0,200.323485,552.002493,16.0
Ayamonte,1671.0,63.0,144.0,21104.0,298.521607,682.335102,8.0
Lepe,2580.0,95.0,188.0,27880.0,340.746055,674.318508,8.0
Aljaraque,1469.0,61.0,149.0,21474.0,284.064450,693.862345,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,159.0,9.0,41.0,1310.0,687.022901,3129.770992,0.0
Cumbres Mayores,108.0,29.0,50.0,1749.0,1658.090337,2858.776444,1.0
Cumbres de Enmedio,1.0,1.0,1.0,51.0,1960.784314,1960.784314,0.0
Corteconcepción,34.0,2.0,10.0,536.0,373.134328,1865.671642,0.0
Cañaveral de León,11.0,4.0,6.0,396.0,1010.101010,1515.151515,0.0
Santa Ana la Real,19.0,1.0,6.0,475.0,210.526316,1263.157895,0.0
Beas,315.0,22.0,52.0,4341.0,506.795669,1197.880673,2.0
Punta Umbría,1111.0,78.0,160.0,15355.0,507.977857,1042.005861,20.0
Villablanca,226.0,11.0,26.0,2885.0,381.282496,901.213172,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Santa Ana la Real,19.0,1.0,6.0,475.0,210.526316,1263.157895,0.0,0.166667
Rosal de la Frontera,43.0,1.0,6.0,1697.0,58.927519,353.565115,0.0,0.166667
Zalamea la Real,106.0,1.0,6.0,3054.0,32.743942,196.463654,0.0,0.166667
Nerva,233.0,5.0,28.0,5169.0,96.730509,541.690849,1.0,0.178571
Corteconcepción,34.0,2.0,10.0,536.0,373.134328,1865.671642,0.0,0.200000
Encinasola,159.0,9.0,41.0,1310.0,687.022901,3129.770992,0.0,0.219512
Galaroza,48.0,1.0,4.0,1382.0,72.358900,289.435601,0.0,0.250000
Valverde del Camino,769.0,13.0,48.0,12750.0,101.960784,376.470588,4.0,0.270833
Higuera de la Sierra,44.0,2.0,7.0,1291.0,154.918668,542.215337,1.0,0.285714
